In [9]:
#from Create_dataset import create_dataset_2_2
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve


#setting the size of the plot
plt.rcParams['figure.figsize'] = [5, 5]

################################################

def create_dataset_2_2(flag,prior1,prior2,sample_size,m01,m02,c0_11,c0_12,c0_21,c0_22,m11,m12,c1_11,c1_12,c1_21,c1_22,seed):
    
    np.random.seed(seed)
    
    classes = 2         #total number of classes 
    num = sample_size      #Sample size
    #initializing mean values and setting up mean values
    mean = np.zeros((2,4)) 
    mean[:,0] = [m01,m02] 
    mean[:,1] = [m11,m12]
    
    cov = np.zeros((2,2,4))
    cov[:,:,0] = np.array([[c0_11,c0_12],[c0_21,c0_22]]) 
    cov[:,:,1] = np.array([[c1_11,c1_12],[c1_21,c1_22]])
    
    # priors for two classes
    class_priors = [prior1, prior2]
    
    #creating a class  labels. first we create uniform dirtribution to create labels and then use those 
    #label distribution to select repective guassian distribution to pick data.
    class_labels = np.zeros((1,num))     #initialization
    #uniform distribution creation
    class_labels[0,:] = (np.random.uniform(0,1,num) >= class_priors[0]).astype(int)
    
    X = np.zeros((classes,num))  #initialization of main dataset
    
    # Data Generation Process: Using uniformly distributed Class labels to pick from respective guassian distribution
    for index in range(num):
        if(class_labels[0,index] == 0):
                X[:,index] = np.random.multivariate_normal(mean[:,0],cov[:,:,0],1)
        else:
                X[:,index] = np.random.multivariate_normal(mean[:,1],cov[:,:,1],1) 
                
    if flag == 1:   
        # Code to Plot the actual distribution
        x00 = [i for i in range(class_labels.shape[1]) if (class_labels[0,i] == 0)]
        x01 = [i for i in range(class_labels.shape[1]) if (class_labels[0,i] == 0)]
        x10 = [i for i in range(class_labels.shape[1]) if (class_labels[0,i] == 1)]
        x11 = [i for i in range(class_labels.shape[1]) if (class_labels[0,i] == 1 )]
        plt.plot(X[0,x00],X[1,x00],'.',color ='g')
        plt.plot(X[0,x11],X[1,x11],'+',color = 'r')
        plt.xlabel("Feature x1")
        plt.ylabel("Feature x2")
        plt.legend(["class 0",'class 1'])
        plt.title("Actual Class distribution")
        plt.show()

    
    return X,class_labels,mean,cov #return the transpose of data array (row matrix)

######################################################################################################
######################################################################################################

def min_P_error_classifier(sample_size,class_prior0,class_prior1,dataset,orig_label,gmean,gcov):
    
    #As it is min P(error) classifer, we will always take 0/1 loss
    loss = np.array([[0,1], [1,0]])
    size = sample_size
    prior = [class_prior0,class_prior1]
    
    mean = np.zeros((2,4)) 
    mean[:,0] = gmean[:,0] 
    mean[:,1] = gmean[:,1]
    
    cov = np.zeros((2,2,4))
    cov[:,:,0] = gcov[:,:,0]
    cov[:,:,1] = gcov[:,:,1]
    
    # Gamma/ threshold
    gamma = ((loss[1,0]-loss[0,0])/(loss[1,0] - loss[1,1])) * (prior[0]/prior[1])
    orig_labels = orig_label

    
    new_labels = np.zeros((1,size))
    # Calculation for discriminant score and decisions
    cond_pdf_class0_log = np.log((multivariate_normal.pdf(dataset.T,mean=mean[:,0],cov = cov[:,:,0])))
    cond_pdf_class1_log = np.log((multivariate_normal.pdf(dataset.T,mean=mean[:,1],cov = cov[:,:,1])))
    
    discriminant_score = cond_pdf_class1_log - cond_pdf_class0_log


    new_labels[0,:] = (discriminant_score >= np.log(gamma)).astype(int)

    # Code to plot the distribution after Classification
    x00 = [i for i in range(new_labels.shape[1]) if (orig_labels[0,i] == 0 and new_labels[0,i] == 0)]
    x01 = [i for i in range(new_labels.shape[1]) if (orig_labels[0,i] == 0 and new_labels[0,i] == 1)]
    x10 = [i for i in range(new_labels.shape[1]) if (orig_labels[0,i] == 1 and new_labels[0,i] == 0)]
    x11 = [i for i in range(new_labels.shape[1]) if (orig_labels[0,i] == 1 and new_labels[0,i] == 1)]
    plt.plot(dataset[0,x00],dataset[1,x00],'.',color ='g')
    plt.plot(dataset[0,x01],dataset[1,x01],'.',color = 'r')
    plt.plot(dataset[0,x11],dataset[1,x11],'+',color ='g')
    plt.plot(dataset[0,x10],dataset[1,x10],'+',color = 'r')
    plt.legend(["class 0 correctly classified",'class 0 wrongly classified','class 1 correctly classified','class 1 wrongly classified'])
    plt.xlabel("Feature x1")
    plt.ylabel("Feature x2")
    plt.title('Distribution after classification')
    plt.show()
    
    
    c0 = np.argwhere(orig_labels[0,:]==0).shape[0]
    c1 = np.argwhere(orig_labels[0,:]==1).shape[0]
    #print("Class 0:",c0)
    #print("Class 1:",c1)
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    tpr = 0
    fpr = 0
    min_TPR = 0
    min_FPR = 0
    TPR = []
    FPR = []
    new_labels1 = np.zeros((1,size))
    d_labels1 = np.zeros((1,size))
    r=map(lambda x: x/10.0,range(0,500))
    print(r)
    for i in r:
        gamma1 = i
        #print(gamma)
        new_labels1[0,:] = (discriminant_score >= np.log(gamma1)).astype(int)
        #d_labels1[0,:] = discriminant_score >= np.log(gamma)
        for i in range(new_labels1.shape[1]): 
            #print("innerforloop")
            if (orig_labels[0,i] == 1 and new_labels1[0,i] == 1):
               TP += 1
            if (orig_labels[0,i] == 0 and new_labels1[0,i] == 1):
               FP += 1
            if (orig_labels[0,i] == 0 and new_labels1[0,i] == 0):
               TN += 1
            if (orig_labels[0,i] == 1 and new_labels1[0,i] == 0):
               FN += 1
        tpr = TP / (TP+FN)
        fpr = FP / (FP+TN)
        TPR.append(tpr)
        FPR.append(fpr)
        if gamma1 == 9.00000:
            min_TPR = tpr
            min_FPR = fpr
        

    plt.plot(FPR,TPR,'-',color = 'r')
    plt.plot(min_FPR,min_TPR, 'g*')
    plt.legend(["ROC Curve",'Min P Error'])
    plt.show()
    plt.close()
    
    
    '''
    #h = .01  # step size in the mesh
    # create a mesh to plot in
    hg = np.linspace(np.floor(min(dataset[:,0])),np.ceil(max(dataset[:,0])),1000);
    vg = np.linspace(np.floor(min(dataset[:,1])),np.ceil(max(dataset[:,1])),1000);
    z = np.zeros((1000,1000))
    xy = np.array(np.meshgrid(hg,vg))
    for i in range(100):
        for j in range(100):
            p1 = multivariate_normal.pdf(np.array(xy[0][i][j],xy[1][i][j]),mean=mean[:,1],cov = cov[:,:,1])
            p2 = multivariate_normal.pdf(np.array(xy[0][i][j],xy[1][i][j]),mean=mean[:,0],cov = cov[:,:,0])
            z[i][j] = np.log(p1) - np.log(p2) - np.log(9)
    
    q00 = [i for i in range(class_labels.shape[1]) if (class_labels[0,i] == 0)]
    q01 = [i for i in range(class_labels.shape[1]) if (class_labels[0,i] == 0)]
    q10 = [i for i in range(class_labels.shape[1]) if (class_labels[0,i] == 1)]
    q11 = [i for i in range(class_labels.shape[1]) if (class_labels[0,i] == 1 )]
    #plt.plot(dataset[0,q00],dataset[1,q00],'.',color ='g')
    #plt.plot(dataset[0,q11],dataset[1,q11],'+',color = 'r')
    #plt.xlabel("Feature x1")
    #plt.ylabel("Feature x2")
    #plt.legend(["class 0",'class 1'])
    #plt.title("Actual Class distribution")
    #plt.show()
    #plt.contour(xy[0], xy[1], z)  
    #, cmap=plt.cm.Paired
    '''
#####################################################################
#####################################################################

def h_fun(z):
    s = 1/(1+np.exp(-z))    
    return s

def param_initial(dim):
    w = np.zeros((dim,1))
    b = 0
    lam0 = np.ones((2,1));
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    return w, b,lam0


###############################

def propagate(w, b, X, Y, lam0):
    m = X.T.shape[1];
    # FORWARD PROPAGATION (FROM X TO COST)
    A = h_fun(np.dot(w.T, X)+b); # compute activation
    print(lam0.shape);
    print(Y.shape);
    print(m.shape);
    print(A.shape);
    cost = -1/m*sum(np.squeeze(lam0[0,0]*Y*np.log(A)+lam0[1,0]*(1-Y)*np.log(1-A)))   # compute cost
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = 1/m*(np.dot(X,(A-Y).T))
    db = 1/m*sum(np.squeeze(A-Y))
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    grads = {"dw": dw,
             "db": db}
    return grads, cost

###############################

def optimize(w, b, X, Y, l, num_iterations, learning_rate, print_cost = False):
    costs = []
    
    for i in range(num_iterations): 
        # Cost and gradient calculation
        grads, cost = propagate(w, b, X, Y, l)
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        # update rule
        w = w-learning_rate*dw
        b = b-learning_rate*db
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    params = {"w": w,
              "b": b}
    grads = {"dw": dw,
             "db": db}
    return params, grads, costs

##############################

def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid(np.dot(w.T,X)+b)  
    for i in range(A.shape[1]):
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if A[:,i] <= 0.5:
            A[:,i]=0
        else:
            A[:,i]=1
    Y_prediction=A
    assert(Y_prediction.shape == (1, m))   
    return Y_prediction
    
#####################################################################
#####################################################################

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    # initialize parameters with zeros
    w, b, l = param_initial(X_train.shape[0])
    # Gradient descent
    #print(w.shape[0])
    #print(b.shape[0])
    #print(w.shape)
    parameters, grads, costs = optimize(w, b, l, X_train, Y_train, num_iterations, learning_rate, print_cost)
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    # Predict test/train set examples
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    return d